In [8]:
from dslmodel.mixins.tools import ToolMixin
# Cell 1: Setup - Initialize GitHub client and required imports
from github import Github, Issue
import netrc

# Read credentials from .netrc
netrc_auth = netrc.netrc()
auth_data = netrc_auth.authenticators('api.github.com')

DEFAULT_REPO = "seanchatmangpt/dslmodel"

global gh
gh = None

if auth_data:
    login, _, password = auth_data
    gh = Github(login, password)
else:
    raise Exception("No authentication information found in ~/.netrc for api.github.com")


class GitHubIssueTool(ToolMixin):
    """Class to manage GitHub Issues"""
    def __init__(self, repository=None):
        super().__init__()
        self.repository = repository or DEFAULT_REPO
        if gh is None:
            raise ValueError("GitHub client not initialized. Please set github_client.")

    def create(self, title: str, body: str):
        """Creates a GitHub Issue"""
        repo = gh.get_repo(self.repository)
        return repo.create_issue(title=title, body=body)
      
    def get(self, number: int):
        """Retrieves a GitHub Issue by number"""
        repo = gh.get_repo(self.repository)
        return repo.get_issue(number)

    def update(self, number: int, **kwargs):
        """Updates a GitHub Issue by number"""
        repo = gh.get_repo(self.repository)
        issue = repo.get_issue(number)
        return issue.edit(**kwargs)

    def close(self, number: int):
        """Closes a GitHub Issue by number"""
        repo = gh.get_repo(self.repository)
        issue = repo.get_issue(number)
        return issue.edit(state='closed')


class GitHubIssueCommentTool:
    """Class to manage GitHub Issue Comments"""
    def __init__(self, issue: Issue = None, issue_id: int = -1):
        if issue is None and issue_id == -1:
            raise ValueError("Either issue or issue_id must be provided")
        
        if issue is None:
            self.issue = gh.get_repo(DEFAULT_REPO).get_issue(issue_id)
        else:
            self.issue = issue
        
        if gh is None:
            raise ValueError("GitHub client not initialized. Please set github_client.")

    def add(self, body: str):
        """Adds a comment to a GitHub Issue"""
        return self.issue.create_comment(body)
    
    def get(self, comment_id: int):
        """Gets a comment from a GitHub Issue"""
        return self.issue.get_comment(comment_id)

    def update(self, comment_id: int, body: str):
        """Updates a comment on a GitHub Issue"""
        return self.issue.edit_comment(comment_id, body)
    
    def delete(self, comment_id: int):
        """Deletes a comment from a GitHub Issue"""
        return self.issue.delete_comment(comment_id)
    
    
    
issue_tool = GitHubIssueTool()


In [12]:
# Cell 2: Create - Create a new GitHub Issue and Comment
new_issue = issue_tool.create("Test Issue", "This is a test issue created from the notebook")
print(f"Issue created: {new_issue.title} (ID: {new_issue.number})")

new_comment = comment_tool.add(new_issue.number, "This is a test comment for the issue")
print(f"Comment created: {new_comment.body} (ID: {new_comment.id})")


Issue created: Test Issue (ID: 4)
Comment created: This is a test comment for the issue (ID: 2406152104)


In [2]:
# Cell 3: Read - Retrieve the created issue and comment
retrieved_issue = issue_tool.get(4)
print(f"Retrieved Issue: {retrieved_issue.title} (Number: {retrieved_issue.number})")


Retrieved Issue: Test Issue (Number: 4)


In [4]:
from dslmodel import init_instant

init_instant()

issue_tool("close issue 4")

reasoning='close issue 4' def_name='close' kwargs={'number': 4}


In [9]:
issue = issue_tool.get(4)
comment_tool = GitHubIssueCommentTool(issue)
comment_tool.get(2406145435)

IssueComment(user=NamedUser(login="seanchatmangpt"), id=2406145435)